In [1]:
# imports
import pdb
from tqdm import tqdm

from IPython.display import display

import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

from fbprophet import Prophet

# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# from dask_ml.preprocessing import StandardScaler as dStandardScaler
# from dask_ml.xgboost import XGBRegressor as dXGBRegressor
# from dask_ml.model_selection import train_test_split as dtrain_test_split

In [2]:
# dask tasks progress bar
pbar = ProgressBar()
pbar.register()

In [3]:
# pd.set_option('display.max_rows', 150)
# pd.set_option('display.max_columns', 50)

# Read Data

In [4]:
# set which columns are metadata and which ones are smart attribuetes
meta_cols = ['date', 'serial_number', 'model', 'capacity_bytes', 'failure']

smart_stats_to_keep = [5, 10, 187, 188, 197, 198]
smart_cols = [f'smart_{i}_raw' for i in smart_stats_to_keep]
smart_cols += [f'smart_{i}_normalized' for i in smart_stats_to_keep]

In [5]:
# read df and keep seagate data
df = dd.read_parquet(
    path='../../../Downloads/backblaze_utils/data_Q3_2020_parquet',
)
df = df[(df['model'].str.startswith('S')) | (df['model'].str.startswith('ZA'))]

In [6]:
# failed vs working serials
failed_serials = df[df['failure']==1]['serial_number'].unique().compute().values
all_serials = df['serial_number'].unique().compute().values
working_serials = np.setdiff1d(all_serials, failed_serials)

[########################################] | 100% Completed | 41.8s
[########################################] | 100% Completed | 41.1s


# Clean Data

Determine which columns have the most missing data, if this missing data is specific to models, and based on this determine which serial numbers data to keep to create a clean dataset

In [7]:
meta_cols = ['date', 'serial_number', 'model', 'capacity_bytes', 'failure']

smart_stats_to_keep = [
    5,
    10,
#     184,
    187,
    188,
    197,
    198,
#     222,
]
smart_cols = [f'smart_{i}_raw' for i in smart_stats_to_keep]
smart_cols += [f'smart_{i}_normalized' for i in smart_stats_to_keep]

## Failed Data

In [18]:
serialwise_featurewise_pct_nans = df[df['serial_number'].isin(failed_serials)][['serial_number']+smart_cols]\
        .groupby('serial_number')\
        .apply(lambda x: x.isna().mean())\
        .compute()
mean_serialwise_pct_nans = serialwise_featurewise_pct_nans.mean(axis=1).sort_values(ascending=False)
mean_serialwise_pct_nans.head(50)

serial_number
7LZ022BP          0.923077
7LZ045HN          0.923077
S2ZYJ9GGB04771    0.307692
S2ZYJ9GGB04761    0.307692
S2ZYJ9FFB18437    0.307692
S2ZYJ9CF504020    0.307692
S2ZYJ9FG404851    0.307692
S2ZYJ9KG303913    0.307692
ZA180XLA          0.014652
ZHZ3SPDW          0.000000
ZA18GX05          0.000000
ZCH080S9          0.000000
ZA12MET8          0.000000
ZA180Q9Q          0.000000
ZJV1BTSY          0.000000
ZA1810R3          0.000000
ZJV5M9KA          0.000000
ZA154CDD          0.000000
ZA1815CT          0.000000
ZA21BZAC          0.000000
Z304J8S5          0.000000
Z304K405          0.000000
ZJV03WEA          0.000000
ZA16Y91F          0.000000
ZA12ADVS          0.000000
ZA14ESZ1          0.000000
ZA181A80          0.000000
Z3054GHK          0.000000
ZA12YWF8          0.000000
ZA16DE0D          0.000000
ZJV4TBZY          0.000000
ZHZ4EZ26          0.000000
Z303R58D          0.000000
ZA141Z0M          0.000000
ZCH0DG24          0.000000
ZHZ3RVZA          0.000000
Z302SQTA      

In [9]:
df[df['serial_number']=='ZA180XLA'][smart_cols].isna().mean().compute().sort_values(ascending=False)

[########################################] | 100% Completed | 40.5s


smart_198_normalized    0.015873
smart_197_normalized    0.015873
smart_188_normalized    0.015873
smart_187_normalized    0.015873
smart_10_normalized     0.015873
smart_5_normalized      0.015873
smart_198_raw           0.015873
smart_197_raw           0.015873
smart_188_raw           0.015873
smart_187_raw           0.015873
smart_10_raw            0.015873
smart_5_raw             0.015873
dtype: float64

In [22]:
failed_ts_df = df[
    df['serial_number'].isin(
        mean_serialwise_pct_nans[mean_serialwise_pct_nans==0].index
    )
][meta_cols + smart_cols].compute()

[########################################] | 100% Completed | 38.4s


In [ ]:
# ret = failed_ts_df[smart_cols].isna().mean()
# ret[(ret!=1) & (ret!=0)].sort_values(ascending=False)

In [ ]:
# too_many_nan_stats = list(set([int(x.split('_')[1]) for x in ret[ret>0.7].index]))
# sorted(too_many_nan_stats)

In [ ]:
# ret = failed_ts_df[['serial_number']+smart_cols].groupby('serial_number').apply(lambda x: x.isna().mean())
# ret

In [ ]:
# ret.mean(axis=1).sort_values(ascending=False).head(50)

In [ ]:
# failed_ts_df[failed_ts_df['serial_number']=='S2ZYJ9GGB04771'][smart_cols].isna().mean().sort_values(ascending=False)

## Working Data

In [ ]:
# ret = working_ts_df[['serial_number']+smart_cols].groupby('serial_number').apply(lambda x: x.isna().mean())
# ret

In [ ]:
# ret.mean(axis=1).sort_values(ascending=False).head(50)

In [17]:
# working_ts_df[working_ts_df['serial_number']=='ZL00564N'][smart_cols].isna().mean().sort_values(ascending=False)

In [23]:
serialwise_featurewise_pct_nans = df[df['serial_number'].isin(working_serials)][['serial_number']+smart_cols]\
        .groupby('serial_number')\
        .apply(lambda x: x.isna().mean())\
        .compute()
mean_serialwise_pct_nans = serialwise_featurewise_pct_nans.mean(axis=1).sort_values(ascending=False)
mean_serialwise_pct_nans.head(50)

[                                        ] | 0% Completed |  0.1s

/home/kachau/.local/share/virtualenvs/ceph_drive_failure-6XiO1Nqh/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning:

`meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result



[########################################] | 100% Completed |  3min 16.1s


serial_number
7LZ01GHG    0.923077
7LZ01NDE    0.923077
7LZ032RA    0.923077
7LZ029KM    0.923077
NB120H9S    0.923077
NB1206GH    0.923077
7QT01D0F    0.923077
7QT01BPW    0.923077
7QT00H3G    0.923077
7LZ047MH    0.923077
7LZ045BC    0.923077
7LZ044SF    0.923077
7LZ036NC    0.923077
7LZ036LY    0.923077
7LZ02ZPX    0.923077
7LZ023NT    0.923077
7QT00H5L    0.923077
7QT00GZN    0.923077
7M00028W    0.923077
7LZ047S2    0.923077
7LZ036H9    0.923077
7LZ026LB    0.923077
7LZ022JQ    0.923077
7LZ01EPC    0.923077
7LZ022PY    0.923077
7LZ036G6    0.923077
7LZ036M8    0.923077
7M000233    0.923077
7QT00FT7    0.923077
7QT01CKQ    0.923077
7QT01DBA    0.923077
NB1205LB    0.923077
7QT00ES6    0.923077
7QT01D2F    0.923077
7LZ023NQ    0.923077
7LZ01N9W    0.923077
7LZ01ENJ    0.923077
W300CTY8    0.923077
NB120KJF    0.923077
NB120JST    0.923077
NB120ARA    0.923077
7QT01CM6    0.923077
7LZ045DD    0.923077
7LZ0440V    0.923077
7LZ036KK    0.923077
7LZ023NS    0.923077
7LZ023LJ    0.923077

In [24]:
working_ts_df = df[
    df['serial_number'].isin(
        mean_serialwise_pct_nans[-int(0.01*len(working_serials)):].index
    )
][meta_cols + smart_cols].compute()

[########################################] | 100% Completed | 39.3s


In [25]:
working_ts_df.shape, failed_ts_df.shape

((95338, 17), (11735, 17))

In [26]:
pbar.unregister()

# Fit Baseline

## fixme: results would have to be shifted -NDAYS_TO_PREDICT coz window size is larger than input data window

In [27]:
# config
Y_COLS = ['smart_5_normalized']
Y_SMART_STATS = set([col.split('_')[1] for col in Y_COLS])
X_COLS = [c for c in smart_cols if c.split('_')[1] not in Y_SMART_STATS]

NDAYS_DATA = 6
NDAYS_TO_PREDICT = 6

In [30]:
def pred_same(curr_ts):
    return ((curr_ts.iloc[-NDAYS_TO_PREDICT] - curr_ts.iloc[-NDAYS_TO_PREDICT-1])**2).mean()

def get_baseline_mses_windowwise(drive_ts):
    # NOTE: assumes only the first y col is to be predicted
    return drive_ts[Y_COLS[0]].rolling(NDAYS_DATA+NDAYS_TO_PREDICT).apply(pred_same)

In [31]:
baseline_mses = failed_ts_df.set_index('date').groupby('serial_number').apply(get_baseline_mses_windowwise)
baseline_mses

serial_number  date      
7LZ022BP       2020-07-01   NaN
               2020-07-02   NaN
               2020-07-03   NaN
               2020-07-04   NaN
               2020-07-05   NaN
                             ..
ZLW0GPC5       2020-07-01   NaN
               2020-07-02   NaN
               2020-07-03   NaN
               2020-07-04   NaN
               2020-07-05   NaN
Name: smart_5_normalized, Length: 12242, dtype: float64

# Fit Model

In [67]:
def pred_prophet(curr_ts):
    return None

def get_prophet_mses_windowwise(drive_ts):
    mses = pd.Series(index=drive_ts.index)
    for start_idx in range(len(drive_ts) - NDAYS_DATA):
        # need to init model every time
        model = Prophet()
        for feat in X_COLS:
            model.add_regressor(feat)
        
        # get previous n days of data for training model
        traindf = drive_ts.iloc[start_idx: start_idx+NDAYS_DATA][X_COLS + Y_COLS]\
                    .reset_index()\
                    .rename(columns={'date': 'ds', f'{Y_COLS[0]}': 'y'})
        
        # if any nans then prophet will complain
        if traindf.isna().any().any():
            continue

        # future to predict
        testdf = drive_ts.iloc[start_idx+NDAYS_DATA: start_idx+NDAYS_DATA+NDAYS_TO_PREDICT][X_COLS]\
                    .reset_index()\
                    .rename(columns={'date': 'ds'})

        # fit and predict
        model.fit(traindf)
        preds = model.predict(testdf)
        
        # calculate error and save it
        error = preds['yhat'].values - drive_ts.iloc[start_idx+NDAYS_DATA: start_idx+NDAYS_DATA+NDAYS_TO_PREDICT][Y_COLS[0]].values
        mses.iloc[start_idx + NDAYS_DATA - 1] = (error**2).mean()
        
    return mses

In [68]:
prophet_mses = failed_ts_df.set_index('date').groupby('serial_number').apply(get_prophet_mses_windowwise)
prophet_mses

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disab

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_chang

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run p

INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_chang

...

INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_chang

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:Disabling yearly seasonality. Run p

INFO:fbprophet:n_changepoints greater than number of observations. Using 3.


ValueError: Found NaN in column 'smart_10_raw'

In [70]:
prophet_mses

NameError: name 'prophet_mses' is not defined

In [69]:
prophet_mses.to_parquet('prophet_mses.parquet')

NameError: name 'prophet_mses' is not defined

In [ ]:
# scalers and models
input_scaler = StandardScaler()
target_scaler = StandardScaler()

adaboost_regr = AdaBoostRegressor()
randomforest_regr = RandomForestRegressor()

In [ ]:
X_COLS_SET = set(X_COLS)
def get_preds(curr_dev_df):
    pdb.set_trace()
    
    # not enough features
    if len(X_COLS.difference(curr_dev_df.columns)) != 0:
        return
    
    preds = [None]*len(curr_dev_df)
    for i in range(len(curr_dev_df) - NDAYS_DATA - NDAYS_TO_PREDICT):

        x = input_scaler.fit_transform(
            curr_dev_df[X_COLS + [Y_COL]].iloc[i: i+NDAYS_DATA+NDAYS_TO_PREDICT]
        )
        y = target_scaler.fit_transform(
            curr_dev_df[[Y_COL]].iloc[i: i+NDAYS_DATA+NDAYS_TO_PREDICT]
        )

        adaboost_regr.fit(
            x[:NDAYS_DATA], 
            y[:NDAYS_DATA].ravel(),
        )

        preds[i+NDAYS_DATA+NDAYS_TO_PREDICT-1] = target_scaler.inverse_transform(
            adaboost_regr.predict(y[np.newaxis, -1])
        )[0]
    return pd.Series(preds, index=curr_dev_df.index)

ret = failed_ts_df.groupby('serial_number').apply(get_preds)
ret.head()

In [ ]:
def get_preds(curr_dev_df):
#     pdb.set_trace()
#     curr_dev_df = curr_dev_df.compute()
    preds = [None]*len(curr_dev_df)
    for i in range(len(curr_dev_df) - NDAYS_DATA):

        x = input_scaler.fit_transform(curr_dev_df[X_COLS + [Y_COL]].iloc[i: i+NDAYS_DATA+1])
        y = target_scaler.fit_transform(curr_dev_df[[Y_COL]].iloc[i: i+NDAYS_DATA+1])

        adaboost_regr.fit(
            x[:NDAYS_DATA], 
            y[:NDAYS_DATA].ravel(),
        )

        preds[i+NDAYS_DATA] = target_scaler.inverse_transform(
            adaboost_regr.predict(data[np.newaxis, -1])
        )[0]
    return pd.Series(preds, index=curr_dev_df.index)

In [ ]:
preds_meta = df._meta.drop([c for c in df.columns if c != Y_COL], axis=1)

ret = df.groupby('serial_number').apply(get_preds, meta=preds_meta).compute()
ret.head()

In [ ]:
preds = [None]*len(device_df)
for i in range(len(device_df) - NDAYS_DATA):
    
    x = input_scaler.fit_transform(device_df[X_COLS + [Y_COL]].iloc[i: i+NDAYS_DATA+1])
    y = target_scaler.fit_transform(device_df[[Y_COL]].iloc[i: i+NDAYS_DATA+1])
    
    adaboost_regr.fit(
        x[:NDAYS_DATA], 
        y[:NDAYS_DATA].ravel(),
    )
    
    preds[i+NDAYS_DATA] = target_scaler.inverse_transform(
        adaboost_regr.predict(data[np.newaxis, -1])
    )[0]
    
preds = pd.Series(preds, index=device_df.index)
preds

In [ ]:
y_true = device_df[y_col]

In [ ]:
sklearn.metrics.mean_squared_error(preds.iloc[NDAYS_DATA+1:], y_true.iloc[NDAYS_DATA+1:])

In [ ]:
xtrain = device_df[x_cols].iloc[:train_idx]
xtest = device_df[x_cols].iloc[train_idx:]
ytrain = device_df[y_col].iloc[:train_idx]
ytest = device_df[y_col].iloc[train_idx:]

In [ ]:
scaler = StandardScaler()
scaler.fit(xtrain)

In [ ]:
# reg = LinearRegression().fit(scaler.transform(xtrain), ytrain)
# reg = GradientBoostingRegressor().fit(scaler.transform(xtrain), ytrain)
reg = RandomForestRegressor().fit(scaler.transform(xtrain), ytrain)
# reg = AdaBoostRegressor().fit(scaler.transform(xtrain), ytrain)
reg.score(scaler.transform(xtest), ytest)

In [ ]:
ytest.plot()
plt.plot(xtest.index, reg.predict(scaler.transform(xtest)))
plt.show()

# Cleanup

In [ ]:
# pbar.unregister()